In [2]:
import os
from langchain_openai.chat_models.azure import AzureChatOpenAI

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

llm = AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=gpt_model,
            )

In [13]:
from langchain_core.prompts import PromptTemplate

def diagram_agent(text):
    "you are a diagram creator expert that generates diagrams with mermaid package"

    prompt = """Your job is to write the code to generate a colorful mermaid diagram describing the text below:

    {text} 
    
    Please understand the text before generating the code. use the text information only , don't use any other information.
    only generate the code as output nothing extra. Do not add code that is not part of mermaid.
    each line in the code must be terminated by ; 
    Code:"""

    template = PromptTemplate.from_template(prompt)
    chain = template | llm
    response = chain.invoke({"text": text})
    return response.content
    

In [10]:
from mermaid import Mermaid

def execute_mermaid(graph):

    if "mermaid" in graph:
        graph = graph.replace("mermaid","").replace("`", "")
    return Mermaid(graph)


In [11]:
import sys
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

#sys.path.append(os.path.abspath("../api/"))

embeddings = AzureOpenAIEmbeddings(
                azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                azure_deployment=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
            )


index_path =  "../api/index/faiss_index"
vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()
template = """Answer questions based only on the following context: 
        
        ### CONTEXT ###
        {context}
        
        Question: {question}
        
        If the context doesn't provide enough information to answer the question, say that you don't know.
        Do not respond with information outside the previous context."""

prompt = PromptTemplate.from_template(template)

chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

query="what python libraries can I use for art generation"
result = chain.invoke(query)

print(result)

You can use the following Python libraries for art generation based on the provided context:

1. **Matplotlib** - A plotting library for creating static, animated, and interactive visualizations.
2. **NumPy** - A library that adds support for large multi-dimensional arrays and matrices, along with mathematical functions to operate on these arrays.
3. **Turtle** - A pre-installed Python library that allows users to create various types of shapes and images, useful for interactive programming.
4. **Pygame** - Mentioned as one of the libraries for creating visuals (specific details not provided in the text).
5. **Processing.py** - Another library mentioned for creating visuals (details not provided).
6. **TensorFlow** - Used for implementing generative adversarial networks (GANs) to create realistic images.

These libraries enable various aspects of generative art creation, from visualizations to interactive and dynamic art pieces.


In [14]:
graph = diagram_agent(text = result)

In [15]:
print(graph)

```mermaid
graph TD;
    A[Python Libraries for Art Generation] --> B[Matplotlib];
    A --> C[NumPy];
    A --> D[Turtle];
    A --> E[Pygame];
    A --> F[Processing.py];
    A --> G[TensorFlow];
    B --> H[Static, Animated, Interactive Visualizations];
    C --> I[Multi-dimensional Arrays, Mathematical Functions];
    D --> J[Shapes and Images Creation];
    E --> K[Visuals Creation];
    F --> L[Visuals Creation];
    G --> M[Generative Adversarial Networks];
    H --> N[Artistic Representations];
    I --> O[Data Processing];
    J --> P[Interactive Programming];
    K --> Q[Dynamic Art Pieces];
    L --> R[Creative Visuals];
    M --> S[Realistic Images];
```


In [17]:
diagram = execute_mermaid(graph)

In [ ]:
diagram

mermaid.mermaid.Mermaid